### ** Only for testing 
#### UNDP SDG Research Assistant

In [9]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

loader = TextLoader("sdg_goals.txt")
docs = loader.load()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embeddings)
retriever = db.as_retriever()

prompt = PromptTemplate(
    template="You are a UNDP research assistant. Context: {context}\n\nQuestion: {question}\n\nAnswer:",
    input_variables=["context", "question"]
)

qa = None
try:
    from langchain.chat_models import init_chat_model
    if openai_key:
        llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0)
        qa = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt}
        )
        #checking if credits are still available
        _ = llm.invoke("ping")
        print("Using OpenAI GPT-4o-mini")
    else:
        raise RuntimeError("OPENAI_API_KEY not set.")
except Exception as e:
    print("OpenAI unavailable or credits exhausted. Falling back to Hugging Face...")
    from langchain_community.llms import HuggingFaceHub
    llm = HuggingFaceHub(
        repo_id="google/flan-t5-base",
        model_kwargs={"temperature": 0.1, "max_length": 256},
        huggingfacehub_api_token=hf_token
    )
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )
    print("Using Hugging Face Flan-T5")

queries = [
    "Which SDG talks about reducing inequality?",
    "What is the goal related to climate change?",
    "How does SDG address gender equality?"
]

for q in queries:
    response = qa.invoke({"query": q})
    print(f"\nQ: {q}")
    print("A:", response["result"])


Using OpenAI GPT-4o-mini

Q: Which SDG talks about reducing inequality?
A: The Sustainable Development Goal (SDG) that talks about reducing inequality is Goal 10: Reduce inequality within and among countries.

Q: What is the goal related to climate change?
A: The goal related to climate change is Goal 13: Take urgent action to combat climate change and its impacts.

Q: How does SDG address gender equality?
A: Sustainable Development Goal 5 (SDG 5) specifically addresses gender equality and aims to achieve gender equality and empower all women and girls. Here are some key aspects of how SDG 5 addresses gender equality:

1. **Elimination of Discrimination**: SDG 5 calls for the elimination of all forms of discrimination against women and girls everywhere, ensuring that they have equal rights and opportunities in all areas of life.

2. **End Violence and Exploitation**: The goal emphasizes the need to end all forms of violence, exploitation, and harmful practices against women and girls, 